PySpark을 로컬 머신에 설치하고 노트북을 사용하기 보다는 머신 러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바 가상머신상의 오브젝트들을 접근할 수 있게 해준다. LocalStandalone Spark을 사용한다.

### 먼저 PySpark과 Py4J를 설치하자

In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9 

In [2]:
!ls -tl sample_data

total 55504
-rw-r--r-- 1 root root 18289443 Jul 16 13:20 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Jul 16 13:20 mnist_train_small.csv
-rw-r--r-- 1 root root   301141 Jul 16 13:20 california_housing_test.csv
-rw-r--r-- 1 root root  1706430 Jul 16 13:20 california_housing_train.csv
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rwxr-xr-x 1 root root      930 Jan  1  2000 README.md


### Spark Session 만들기
**Spark Session** : Spark Session은 Spark 2.0부터 엔트리포이트로 사용한다. 그 이전에는 SparkContext가 사용되었다. Spark Sessiond을 이용해 RDD, 데이터 프레임 등을 만든다. Spark Session은 Spark Session.builder를 호출하여 생성하며 다양한 함수들을 통해 세부 설정이 가능하다.

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("PySpark Tutorial") \
    .getOrCreate()

# spark = SparkSession \
#     .builder \
#     .master("local[*]") \           ## 사용하고싶은 SparkCluster이름[사용하고 싶은 cpu의 개수]
#     .appName("PySpark Tutorial") \
#     .getOrCreate()

In [4]:
spark

### Python 객체를 RDD로 변환해보기

#### 1. Python 리스트 생성

In [5]:
name_list_json = ['{"name" : "keeyong"}', '{"name" : "benjamin"}', '{"name" : "claire"}']

In [6]:
for n in name_list_json :
    print(n)

{"name" : "keeyong"}
{"name" : "benjamin"}
{"name" : "claire"}


In [7]:
import json

for n in name_list_json :
    jn = json.loads(n)
    print(jn["name"])

keeyong
benjamin
claire


#### 2. 파이썬 리스트를 RDD로 변환.  
- RDD로 변환되는 순간 Spark 클러스터의 서버들에 데이터가 나눠 저장됨(파티션).  
- 또한 Lazy Execution이 된다는 점 기억 (액션을 취하기 전까지는 spark가 실행이 안됨)

In [8]:
rdd = spark.sparkContext.parallelize(name_list_json)

In [9]:
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262

In [10]:
rdd.count()

3

In [11]:
# map : rdd의 레코드들을 다음 액션을 통해 새로운 rdd 객체 생성
# string이었던 rdd의 원소들이 python의 dictionary형으로 바뀜
parsed_rdd = rdd.map(lambda el:json.loads(el))  

In [12]:
parsed_rdd

PythonRDD[2] at RDD at PythonRDD.scala:53

In [13]:
parsed_rdd.collect()  # rdd가 Python 프로그래밍으로 넘어옴

[{'name': 'keeyong'}, {'name': 'benjamin'}, {'name': 'claire'}]

In [14]:
parsed_name_rdd = rdd.map(lambda el:json.loads(el)["name"])

In [15]:
parsed_name_rdd.collect()   # 받아오는 데이터가 너무 커지면 안됨. 못 받아옴 따로 어딘가에 저장해야함

['keeyong', 'benjamin', 'claire']

### 파이썬 리스트를 데이터프레임으로 변환하기

In [16]:
from pyspark.sql.types import StringType

df = spark.createDataFrame(name_list_json, StringType())

In [17]:
df.count()

3

In [18]:
df.printSchema()   # 기본적으로 필드 이름이 value가 됨

root
 |-- value: string (nullable = true)



In [19]:
df.select('*').collect()  # '*'는 모든 레코드

[Row(value='{"name" : "keeyong"}'),
 Row(value='{"name" : "benjamin"}'),
 Row(value='{"name" : "claire"}')]

In [20]:
df.select('value').collect()

[Row(value='{"name" : "keeyong"}'),
 Row(value='{"name" : "benjamin"}'),
 Row(value='{"name" : "claire"}')]

In [21]:
from pyspark.sql import Row

row = Row("name")  # Or some other column name
df_name = parsed_name_rdd.map(row).toDF()  # row라는 필드를 주면서 DF로 바꿈

In [22]:
df_name.printSchema()  # 필드 이름이 name으로 바뀜

root
 |-- name: string (nullable = true)



In [23]:
df_name.select('name').collect()

[Row(name='keeyong'), Row(name='benjamin'), Row(name='claire')]